In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import f_oneway, chi2_contingency
from pathlib import Path

# Paths
IN_PATH = Path("../data/processed/02_clustered_data.csv")

# Load clustered, scaled data from Notebook 02
df = pd.read_csv(IN_PATH)
if "Cluster" not in df.columns:
    raise RuntimeError("Cluster column missing. Run 02_pca_clustering.ipynb first.")

# Top biomarkers to test (edit as needed)
top_biomarkers = [
    'Proteina C reativa mg/dL',
    'Leukocytes',
    'Platelets',
    'Creatinine',
    'Urea',
]

results = []
for feat in top_biomarkers:
    if feat not in df.columns:
        print(f"Skipping missing feature: {feat}")
        continue
    vals = pd.to_numeric(df[feat], errors='coerce')
    df_feat = pd.DataFrame({'Cluster': df['Cluster'], 'val': vals}).dropna()
    groups = [df_feat.loc[df_feat['Cluster'] == c, 'val'] for c in sorted(df_feat['Cluster'].unique())]
    if any(len(g) == 0 for g in groups):
        print(f"Skipping {feat}: empty group")
        continue
    f_stat, p_val = f_oneway(*groups)
    results.append({
        'Feature': feat,
        'F_stat': f_stat,
        'p_value': p_val,
        'Significant?': '*' if p_val < 0.05 else ''
    })

if results:
    evidence_df = pd.DataFrame(results).sort_values(by='p_value')
    evidence_df['p_value_fmt'] = evidence_df['p_value'].apply(lambda p: '< 0.001' if p < 1e-3 else f"{p:.3g}")
    evidence_df = evidence_df[['Feature', 'F_stat', 'p_value_fmt', 'Significant?']]
    print("One-way ANOVA (top biomarkers):")
    display(evidence_df)
else:
    print("No ANOVA results computed. Check feature availability.")

# Chi-square for ICU admission and ICU percentages by cluster
icu_col = 'Patient addmited to intensive care unit (1=yes, 0=no)'
if icu_col in df.columns:
    contingency = pd.crosstab(df['Cluster'], df[icu_col].fillna(0).astype(int))
    chi2, chi_p, _, _ = chi2_contingency(contingency)
    print(f"Chi-square p-value for Cluster vs ICU: {chi_p:.4g}")
    display(contingency)

    icu_rates = (contingency.div(contingency.sum(axis=1), axis=0) * 100).round(2)
    print("ICU admission rate (%) by cluster:")
    display(icu_rates[[c for c in icu_rates.columns if c == 1 or c == '1' or c == 1.0]])
else:
    print(f"ICU column not found: {icu_col}")



One-way ANOVA (top biomarkers):


,Feature,F_stat,p_value_fmt,Significant?
4,Urea,146.672172,< 0.001,*
3,Creatinine,49.458903,< 0.001,*
1,Leukocytes,24.800230,< 0.001,*
0,Proteina C reativa mg/dL,6.597805,0.00146,*
2,Platelets,6.361135,0.00185,*


Chi-square p-value for Cluster vs ICU: 2.134e-13


"Patient addmited to intensive care unit (1=yes, 0=no)",0,4
Cluster,,
0,555,20
1,0,1
2,19,8


ICU admission rate (%) by cluster:


"Patient addmited to intensive care unit (1=yes, 0=no)"
Cluster
0
1
2


### Clinical validation takeaway
Statistical Validation confirms that the 3 phenotypes are distinct (p < 0.001). The “Multi-System Failure” phenotype shows a significantly higher risk of ICU admission compared to the “Stable” group, validating the clinical utility of the model.

